In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

In [ ]:
!pwd

In [ ]:
!ls

In [ ]:
!apt-get install unzip

In [ ]:
# !unzip "/content/drive/MyDrive/Cancer_cn.zip" -d "/content/drive/MyDrive/Cancer_cn"

In [ ]:
# Load the data
data_dir = "/content/drive/MyDrive/Cancer_cn/Brain-Tumor-Classification-DataSet-master/Training"
img_height = 200
img_width = 200
batch_size = 200
normlazation_size = 280.0
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=200,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
class_names = train_ds.class_names
#print(class_names)

In [ ]:
from tensorflow import data
AUTOTUNE = data.experimental.AUTOTUNE
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./normlazation_size)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

In [ ]:
num_classes = 4

model = tf.keras.Sequential([
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.MaxPooling2D(),
  tf.keras.layers.Conv2D(32, 3, activation='relu'),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(num_classes)
])
model.compile(
  optimizer=tf.keras.optimizers.Adam(),
  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [ ]:
history=model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=10
)

In [ ]:
model.evaluate(train_ds)

In [ ]:
pd.DataFrame(history.history).plot()

In [ ]:
# lr_scheduler=tf.keras.callbacks.LearningRateScheduler(lambda epoch:1e-3*10**(epoch/20))
# find_lr_history=model.fit(train_ds,epochs=10,callbacks=[lr_scheduler])

In [ ]:
# #plot the learning rate decay curve
# import numpy as np
# import matplotlib.pyplot as plt
# lrs=1e-3*(10**(tf.range(10)/20))
# plt.semilogx(lrs,find_lr_history.history["loss"])
# plt.xlabel("Learning rate")
# plt.ylabel("Loss")
# plt.title("Finding the ideal learning rate")


In [ ]:
def evaluate_model(model, dataset, class_names):
    # Initialize an empty list to store the results
    results = []

    # Initialize empty lists to store true labels and predicted labels
    true_labels = []
    predicted_labels = []

    # Iterate over all batches in the dataset
    for image_batch, labels_batch in dataset:
        # Get predictions for the current batch
        predictions = model.predict(image_batch)

        # Convert labels_batch to a NumPy array
        labels_batch_np = labels_batch.numpy()

        # Store true labels and predicted labels
        true_labels.extend(labels_batch_np)
        predicted_labels.extend(predictions.argmax(axis=1))

        # Compare predictions with true labels and store the results
        for i in range(len(labels_batch_np)):
            comparison = predictions.argmax(axis=1)[i] == labels_batch_np[i]
            results.append(comparison)

    # Convert the results to a DataFrame
    results_df = pd.DataFrame({
        'True Label': true_labels,
        'Predicted Label': predicted_labels,
        'Comparison': results
    })

    # Export the DataFrame to a CSV file
    results_df.to_csv('results.csv', index=False)

    # Calculate and print the total accuracy
    accuracy = np.mean(results)
    print(f'Total Accuracy: {accuracy * 100:.2f}%')

    # Plot the predictions with the labels and accuracy
    plt.figure(figsize=(10, 10))
    for images, labels in dataset.take(1):
        for i in range(9):
            ax = plt.subplot(3, 3, i + 1)
            plt.imshow(images[i].numpy().astype("uint8"))
            plt.title(f"True: {class_names[labels[i]]}\nPred: {class_names[predicted_labels[i]]}")
            plt.axis("off")
    plt.show()

    return results_df, accuracy

# Call the function with the model and dataset
#results_df, accuracy = evaluate_model(model, train_ds, class_names)

In [ ]:
# model.fit(
#   image_batch,labels_batch,
#   epochs=7
# )

In [ ]:
# results_df, accuracy = evaluate_model(model, train_ds, class_names)

In [ ]:
# Load the data
data_dir = "/content/drive/MyDrive/Cancer_cn/Brain-Tumor-Classification-DataSet-master/Testing"
img_height = 200
img_width = 200
batch_size = 374
test_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
normalization_layer = tf.keras.layers.Rescaling(1./normlazation_size)
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

In [ ]:
results_df, accuracy = evaluate_model(model, test_ds, class_names)

In [ ]:
#tf.keras.models.save_model(model, 'model.keras')

In [ ]:
#model2=tf.keras.models.load_model('model.keras')

In [ ]:
#results_df, accuracy = evaluate_model(model2, test_ds, class_names)

In [ ]:
# results = []

# # Initialize empty lists to store true labels and predicted labels
# true_labels = []
# predicted_labels = []

# # Iterate over all batches in the dataset
# for image_batch, labels_batch in test_ds:
#   # Get predictions for the current batch
#   predictions = model.predict(image_batch)

#   # Convert labels_batch to a NumPy array
#   labels_batch_np = labels_batch.numpy()

#   # Store true labels and predicted labels
#   true_labels.extend(labels_batch_np)
#   predicted_labels.extend(predictions.argmax(axis=1))

#   # Compare predictions with true labels and store the results
#   for i in range(len(labels_batch_np)):
#       comparison = predictions.argmax(axis=1)[i] == labels_batch_np[i]
#       results.append(comparison)
#       plt.imshow(image_batch[i].numpy().astype("uint8"))
#       plt.title(f"True: {class_names[labels_batch_np[i]]}\nPred: {class_names[predictions.argmax(axis=1)[i]]}")
#       plt.axis("off")
#       plt.show()


In [ ]:
from sklearn.metrics import confusion_matrix
for images, lables_batch in test_ds:
  predictions=model.predict(images)
  predictions=predictions.argmax(axis=1)
confusion_matrix(lables_batch,predictions)

In [ ]:
# predictions=model.predict(images)
# predictions=predictions.argmax()#that's why we need axis=1
# predictions

In [ ]:
import itertools
from sklearn.metrics import confusion_matrix
def plot_confusion_matrix(y_true, y_preds, classes=None, figsize=(10, 10), text_size=15, label_size=20, marker_size=10):
    cm = confusion_matrix(y_true, tf.round(y_preds))
    cm_norm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]
    n_classes = cm.shape[0]

    fig, ax = plt.subplots(figsize=figsize)
    cax = ax.matshow(cm, cmap=plt.cm.Blues)
    fig.colorbar(cax)

    if classes:
        labels = classes
    else:
        labels = np.arange(cm.shape[0])

    ax.set(title="Confusion Matrix",
           xlabel="Predicted Label",
           ylabel="True Label",
           xticks=np.arange(n_classes),
           yticks=np.arange(n_classes),
           xticklabels=labels,
           yticklabels=labels)

    ax.xaxis.set_label_position("bottom")
    ax.xaxis.tick_bottom()
    ax.yaxis.label.set_size(label_size)
    ax.xaxis.label.set_size(label_size)
    ax.tick_params(axis='x', labelsize=marker_size)
    ax.tick_params(axis='y', labelsize=marker_size)

    threshold = (cm.max() + cm.min()) / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, f"{cm[i, j]} ({cm_norm[i, j] * 100:.1f}%)",
                 horizontalalignment="center",
                 color="white" if cm[i, j] > threshold else "black",
                 size=text_size)

In [ ]:
for images, lables in test_ds:
  print(images.shape)
  print(lables.shape)
  print(image_batch.shape)
  predictions=model.predict(images)
  predictions=predictions.argmax(axis=1)
  print(predictions.shape)
  labels=lables
plot_confusion_matrix(labels, predictions, classes=(test_ds.class_names), figsize=(10, 10),text_size=12,label_size=6,marker_size=13)